<a href="https://colab.research.google.com/github/Gauthamsv7/NLP_BERT/blob/main/BERT_Tesla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import BertTokenizer
import torch
import numpy as np

In [11]:
model  = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
tesla_car = """
Tesla, Inc is an American multinational automotive and clean energy company.
Headquartered in Austin, Texas, it designs, manufactures and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale,
solar panels and solar shingles, and related products and services.
Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors.
Its name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, Elon Musk joined as Tesla's largest shareholder;
in 2008, he was named chief executive officer. In 2008, the company began production of its first car model, the Roadster sports car,
followed by the Model S sedan in 2012, the Model X SUV in 2015, the Model 3 sedan in 2017, the Model Y crossover in 2020,
the Tesla Semi truck in 2022 and the Cybertruck pickup truck in 2023. The Model 3 is the all-time best-selling plug-in electric car worldwide,
 and in June 2021 became the first electric car to sell 1 million units globally.[6] In 2023, the Model Y was the best-selling vehicle, of any kind, globally.[3]
Tesla is one of the world's most valuable companies in terms of market capitalization. In October 2021, Tesla temporarily became a trillion dollar company,
the seventh U.S. company to do so. In 2023, the company led the battery electric vehicle market, with 19.9% share.
Also in 2023, the company was ranked 69th in the Forbes Global 2000.[7] As of March 2024, it is the world's most valuable automaker.
Tesla has been the subject of lawsuits, government scrutiny, and journalistic criticism, stemming from allegations of whistleblower retaliation,
worker rights violations, safety issues, product defects, fraud, and Musk's many controversial statements.
The company was incorporated as Tesla Motors, Inc.
on July 1, 2003, by Martin Eberhard and Marc Tarpenning.[8][9] They served as chief executive officer and chief financial officer, respectively.
[10] Eberhard said that he wanted to build "a car manufacturer that is also a technology company", with its core technologies as "the battery, the computer software, and the proprietary motor".[11]
Ian Wright was Tesla's third employee, joining a few months later.[8]"""


In [13]:
def bert_question_answer(question,passage,max_len=1500):

  input_ids = tokenizer.encode(question,passage,max_length=max_len,padding=True,truncation=True,add_special_tokens = True)


  sep_index = input_ids.index(102)
  len_question = sep_index + 1
  len_passage = len(input_ids) - len_question

  segment_ids = [0] * len_question + [1] * len_passage

  tokens = tokenizer.convert_ids_to_tokens(input_ids)

  start_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))[0]
  end_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))[1]

  start_token_scores = start_token_scores.detach().numpy().flatten()
  end_token_scores = end_token_scores.detach().numpy().flatten()

  answer_start_index = np.argmax(start_token_scores)
  answer_end_index = np.argmax(end_token_scores)

  start_token_score = np.round(start_token_scores[answer_start_index],2)
  end_token_score = np.round(end_token_scores[answer_end_index],2)

  answer = tokens[answer_start_index]
  for i in range(answer_start_index + 1, answer_end_index + 1):
    if tokens[i][0:2] == '##':
      answer += tokens[i][2:]
    else:
      answer += ' ' + tokens[i]

  if ( answer_start_index == 0) or (start_token_score < 0) or (answer == "[SEP]") or (answer_end_index < answer_start_index):
    answer = "Sorry Answer Not Found"

  return answer, start_token_score, end_token_score, answer_start_index, answer_end_index



In [14]:
print(len(tesla_car.split()))

351


In [15]:
q1 = "When is Tesla incorporated"
print("Question : ",q1)

ans,_,_,_,_ = bert_question_answer(q1,tesla_car)

print("Answer : ",ans,"\n")


Question :  When is Tesla incorporated
Answer :  july 1 , 2003 



In [16]:
#@title Tesla Question Answering Application { vertical-output:true}
#@markdown ---
passage = "Tesla, Inc is an American multinational automotive and clean energy company.  Headquartered in Austin, Texas, it designs, manufactures and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale,  solar panels and solar shingles, and related products and services. Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors.  Its name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, Elon Musk joined as Tesla's largest shareholder;  in 2008, he was named chief executive officer. In 2008, the company began production of its first car model, the Roadster sports car,  followed by the Model S sedan in 2012, the Model X SUV in 2015, the Model 3 sedan in 2017, the Model Y crossover in 2020,  the Tesla Semi truck in 2022 and the Cybertruck pickup truck in 2023. The Model 3 is the all-time best-selling plug-in electric car worldwide,  and in June 2021 became the first electric car to sell 1 million units globally.[6] In 2023, the Model Y was the best-selling vehicle, of any kind, globally.[3] Tesla is one of the world's most valuable companies in terms of market capitalization. In October 2021, Tesla temporarily became a trillion dollar company,  the seventh U.S. company to do so. In 2023, the company led the battery electric vehicle market, with 19.9% share.  Also in 2023, the company was ranked 69th in the Forbes Global 2000.[7] As of March 2024, it is the world's most valuable automaker. Tesla has been the subject of lawsuits, government scrutiny, and journalistic criticism, stemming from allegations of whistleblower retaliation,  worker rights violations, safety issues, product defects, fraud, and Musk's many controversial statements. The company was incorporated as Tesla Motors, Inc. on July 1, 2003, by Martin Eberhard and Marc Tarpenning.[8][9] They served as chief executive officer and chief financial officer, respectively. [10] Eberhard said that he wanted to build \"a car manufacturer that is also a technology company\", with its core technologies as \"the battery, the computer software, and the proprietary motor\".[11] Ian Wright was Tesla's third employee, joining a few months later.[8]" #@param {type:"string"}
question = "who is the founder of Tesla ? " #@param {type:"string"}
#@markdown ---

answer,_,_,_,_ = bert_question_answer(question,passage)

#@markdown Answer : {}
print("Answer : ",answer)


Answer :  martin eberhard and marc tarpenning as tesla motors . its name is a tribute to inventor and electrical engineer nikola tesla
